<a href="https://colab.research.google.com/github/selenayack/GanDeepLearningg/blob/main/bili%C5%9Fim_teknolojisi_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Google Drivea Bağlandık



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


zip dosyaıs

In [ ]:
!cp /content/drive/MyDrive/LOLdataset.zip "/content"

In [ ]:
!unzip LOLdataset.zip -d /content/dataset

Archive:  LOLdataset.zip
   creating: /content/dataset/eval15/
   creating: /content/dataset/eval15/high/
  inflating: /content/dataset/eval15/high/748.png  
   creating: /content/dataset/__MACOSX/
   creating: /content/dataset/__MACOSX/eval15/
   creating: /content/dataset/__MACOSX/eval15/high/
  inflating: /content/dataset/__MACOSX/eval15/high/._748.png  
  inflating: /content/dataset/eval15/high/665.png  
  inflating: /content/dataset/__MACOSX/eval15/high/._665.png  
  inflating: /content/dataset/eval15/high/111.png  
  inflating: /content/dataset/__MACOSX/eval15/high/._111.png  
  inflating: /content/dataset/eval15/high/493.png  
  inflating: /content/dataset/__MACOSX/eval15/high/._493.png  
  inflating: /content/dataset/eval15/high/22.png  
  inflating: /content/dataset/__MACOSX/eval15/high/._22.png  
  inflating: /content/dataset/eval15/high/23.png  
  inflating: /content/dataset/__MACOSX/eval15/high/._23.png  
  inflating: /content/dataset/eval15/high/669.png  
  inflating: /con

Resimleri png/jpeg/jpg'den npz formatına (png2npz.py) dönüştürün

In [ ]:
################################ Imports ####################################
import shutil
from os import listdir
from tensorflow.keras.utils import load_img, img_to_array
from numpy import asarray, savez_compressed


################################ Load Dataset ####################################

def load_images(path, size=(256, 256)):
    data_list = list()
    # enumerate filenames in directory, assume all are images
    for filename in listdir(path):
        if filename.endswith('.png'):  # Check if filename ends with ".png"
            # load and resize the image
            pixels = load_img(path + filename, target_size=size)
            # convert to numpy array
            pixels = img_to_array(pixels)
            # store
            data_list.append(pixels)

    return asarray(data_list)


################################ Main Function ####################################
def main():
    # load images
    path = "/content/dataset/our485/"
    high = load_images(path + 'high/')  # directory "path/ground_truth" should be present
    low = load_images(path + 'low/')  # directory "path/low" should be present

    savez_compressed('dataset1.npz', high, low)


if __name__ == '__main__':
    main()


# Yeni Bölüm

Kitaplıkları İçe Aktarma

In [ ]:
import os
import time
import datetime
import tensorflow as tf
import matplotlib.pyplot as plt
from numpy.random import randint
from tensorflow.keras import Input
from numpy import load, zeros, ones
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.layers import Conv2D, UpSampling2D, LeakyReLU, Activation
from tensorflow.keras.layers import Concatenate, Dropout, BatchNormalization, LeakyReLU


Defining Discriminator  basit bir sınıflandırıcıdır. Gerçek verileri generator tarafından oluşturulan verilerden ayırt etmeye çalışır.

discriminator, hem gerçek verileri hem de generatorden alınan sahte verileri sınıflandırır.

In [ ]:
def define_discriminator(image_shape):
	init = RandomNormal(stddev=0.02)

	in_src_image = Input(shape=image_shape)
	in_target_image = Input(shape=image_shape)

	merged = Concatenate()([in_src_image, in_target_image])

	d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
	d = LeakyReLU(alpha=0.2)(d)

	d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)

	d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)

	d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)

	d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)

	d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
	patch_out = Activation('sigmoid')(d)

	model = Model([in_src_image, in_target_image], patch_out)

	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
	return model

Generator bölümü  gerçeğe benzeyen yeni veriler (resimler, sesler, modeller vb.) üretir

Bu iki sinir ağı birbiriyle çekişirken Ayırt Edici ağ giderek gerçek ve sahte resimleri daha iyi ayırt etmeye başlar. Üretici ağ ise daha gerçekçi sahte resimler üretir.

In [ ]:
def define_generator(image_shape = (256, 256, 3)):
    init = RandomNormal(stddev=0.02)
    in_image = Input(shape=image_shape)

    g = Conv2D(64, (7,7), padding='same', kernel_initializer=init)(in_image)
    g = BatchNormalization()(g, training=True)
    g3 = LeakyReLU(alpha=0.2)(g)

    g = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g3)
    g = BatchNormalization()(g, training=True)
    g2 = LeakyReLU(alpha=0.2)(g)

    g = Conv2D(256, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g2)
    g = BatchNormalization()(g, training=True)
    g1 = LeakyReLU(alpha=0.2)(g)

    for _ in range(6):
        g = Conv2D(256, (3,3), padding='same', kernel_initializer=init)(g1)
        g = BatchNormalization()(g, training=True)
        g = LeakyReLU(alpha=0.2)(g)

        g = Conv2D(256, (3,3), padding='same', kernel_initializer=init)(g)
        g = BatchNormalization()(g, training=True)

        g1 = Concatenate()([g, g1])

    g = UpSampling2D((2, 2))(g1)
    g = Conv2D(128, (1, 1), kernel_initializer=init)(g)
    g = Dropout(0.5)(g, training=True)
    g = Concatenate()([g, g2])
    g = BatchNormalization()(g, training=True)
    g = LeakyReLU(alpha=0.2)(g)

    g = UpSampling2D((2, 2))(g)
    g = Conv2D(64, (1, 1), kernel_initializer=init)(g)
    g = Dropout(0.5)(g, training=True)
    g = Concatenate()([g, g3])
    g = BatchNormalization()(g, training=True)
    g = LeakyReLU(alpha=0.2)(g)

    g = Conv2D(3, (7,7), padding='same', kernel_initializer=init)(g)
    g = BatchNormalization()(g, training=True)
    out_image = Activation('tanh')(g)

    model = Model(in_image, out_image)
    return model

GAN EĞİTİMİ BAŞLIYOR

In [ ]:
def define_gan(g_model, d_model, image_shape):
	# make weights in the discriminator not trainable
	for layer in d_model.layers:
		if not isinstance(layer, BatchNormalization):
			layer.trainable = False
	# define the source image
	in_src = Input(shape=image_shape)
	# connect the source image to the generator input
	gen_out = g_model(in_src)
	# connect the source input and generator output to the discriminator input
	dis_out = d_model([in_src, gen_out])
	# src image as input, generated image and classification output
	model = Model(in_src, [dis_out, gen_out])
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
	return model

Gerçek Örnekleri Yükleme

In [ ]:
def load_real_samples(filename):
	# load compressed arrays
	data = load(filename)
	# unpack arrays
	X1, X2 = data['arr_0'], data['arr_1']
	# scale from [0,255] to [-1,1]
	X1 = (X1 - 127.5) / 127.5
	X2 = (X2 - 127.5) / 127.5
	return [X2, X1]

Gerçek Sahte Örnekler Oluşturma

In [ ]:
def generate_real_samples(dataset, n_samples, patch_shape):
	# unpack dataset
	trainA, trainB = dataset
	# choose random instances
	ix = randint(0, trainA.shape[0], n_samples)
	# retrieve selected images
	X1, X2 = trainA[ix], trainB[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, patch_shape, patch_shape, 1))
	return [X1, X2], y


Sahte Örnekler Oluşturma

In [ ]:

def generate_fake_samples(g_model, samples, patch_shape):
	# generate fake instance
	X = g_model.predict(samples)
	# create 'fake' class labels (0)
	y = zeros((len(X), patch_shape, patch_shape, 1))
	return X, y

Eğitim modelini özetleme ve modeli kaydetme


In [ ]:
def summarize_performance(step, g_model, d_model, dataset, n_samples=3):
    # select a sample of input images
    [X_realA, X_realB], _ = generate_real_samples(dataset, n_samples, 1)
    # generate a batch of fake samples
    X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
    # scale all pixels from [-1,1] to [0,1]
    X_realA = (X_realA + 1) / 2.0
    X_realB = (X_realB + 1) / 2.0
    X_fakeB = (X_fakeB + 1) / 2.0
    # plot real source images
    plt.figure(figsize=(14, 14))
    for i in range(n_samples):
        plt.subplot(3, n_samples, 1 + i)
        plt.axis('off')
        plt.title('Low-Light')
        plt.imshow(X_realA[i])
    # plot generated target image
    for i in range(n_samples):
        plt.subplot(3, n_samples, 1 + n_samples + i)
        plt.axis('off')
        plt.title('Generated')
        plt.imshow(X_fakeB[i])
    # plot real target image
    for i in range(n_samples):
        plt.subplot(3, n_samples, 1 + n_samples*2 + i)
        plt.axis('off')
        plt.title('Ground Truth')
        plt.imshow(X_realB[i])
    # save plot to file
    filename1 = step_output + 'plot_%06d.png' % (step+1)
    plt.savefig(filename1)
    plt.close()
    # save the generator model
    filename2 = model_output + 'gen_model_%06d.h5' % (step+1)
    g_model.save(filename2)
    # save the discriminator model
    filename3 = model_output + 'disc_model_%06d.h5' % (step+1)
    d_model.save(filename3)
    print('[.] Saved Step : %s' % (filename1))
    print('[.] Saved Model: %s' % (filename2))
    print('[.] Saved Model: %s' % (filename3))

EĞİTİM


In [ ]:
def train(d_model, g_model, gan_model, dataset, n_epochs=100, n_batch=12):
    # determine the output square shape of the discriminator
    n_patch = d_model.output_shape[1]
    # unpack dataset
    trainA, trainB = dataset
    # calculate the number of batches per training epoch
    bat_per_epo = 1
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    print("[!] Number of steps {}".format(n_steps))
    print("[!] Saves model/step output at every {}".format(bat_per_epo * 1))
    # manually enumerate epochs
    for i in range(n_steps):
        start = time.time()
        # select a batch of real samples
        [X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
        # generate a batch of fake samples
        X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
        # update discriminator for real samples
        d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
        # update discriminator for generated samples
        d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
        # update the generator
        g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB])
        # summarize performance
        time_taken = time.time() - start
        print(
            '[*] %06d, d1[%.3f] d2[%.3f] g[%06.3f] ---> time[%.2f], time_left[%.08s]'
                %
            (i+1, d_loss1, d_loss2, g_loss, time_taken, str(datetime.timedelta(seconds=((time_taken) * (n_steps - (i + 1))))).split('.')[0].zfill(8))
        )
        # summarize model performance
        if (i+1) % (bat_per_epo * 1) == 0:
            summarize_performance(i, g_model, d_model, dataset)

Main Function

Loading Dataset

In [ ]:

dataset = load_real_samples('/content/dataset1.npz')
print('Loaded', dataset[0].shape, dataset[1].shape)
image_shape = dataset[0].shape[1:]

Loaded (485, 256, 256, 3) (485, 256, 256, 3)


In [ ]:
tanımlayıcı modeller

Defining Models

In [ ]:
d_model = define_discriminator(image_shape)
g_model = define_generator(image_shape)
gan_model = define_gan(g_model, d_model, image_shape)

/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Model Dizini Oluşturma ve train İşlevini Çağırma

In [ ]:
import shutil
import os
dir = '/content/'
fileName = 'Enhancement Model'
if os.path.exists(dir+fileName):
    shutil.rmtree(dir+fileName)
    print(f"Directory '{dir}' deleted successfully.")
else:
    print(f"Directory '{dir}' does not exist.")
step_output = dir + fileName + "/Step Output/"
model_output = dir + fileName + "/Model Output/"
if fileName not in os.listdir(dir):
    os.mkdir(dir + fileName)
    os.mkdir(step_output)
    os.mkdir(model_output)

train(d_model, g_model, gan_model, dataset,  n_batch=12)

Directory '/content/' does not exist.
[!] Number of steps 100
[!] Saves model/step output at every 1
1/1 [==============================] - 17s 17s/step
[*] 000001, d1[0.468] d2[2.138] g[57.706] ---> time[61.31], time_left[01:41:09]
1/1 [==============================] - 3s 3s/step


[.] Saved Step : /content/Enhancement Model/Step Output/plot_000001.png
[.] Saved Model: /content/Enhancement Model/Model Output/gen_model_000001.h5
[.] Saved Model: /content/Enhancement Model/Model Output/disc_model_000001.h5
1/1 [==============================] - 0s 32ms/step
[*] 000002, d1[2.404] d2[2.514] g[43.200] ---> time[2.17], time_left[00:03:32]
1/1 [==============================] - 0s 34ms/step


[.] Saved Step : /content/Enhancement Model/Step Output/plot_000002.png
[.] Saved Model: /content/Enhancement Model/Model Output/gen_model_000002.h5
[.] Saved Model: /content/Enhancement Model/Model Output/disc_model_000002.h5
1/1 [==============================] - 0s 51ms/step
[*] 000003, d1[0.390] d2[0.785] g[34.836] ---> time[1.95], time_left[00:03:09]
1/1 [==============================] - 0s 22ms/step


[.] Saved Step : /content/Enhancement Model/Step Output/plot_000003.png
[.] Saved Model: /content/Enhancement Model/Model Output/gen_model_000003.h5
[.] Saved Model: /content/Enhancement Model/Model Output/disc_model_000003.h5
1/1 [==============================] - 0s 31ms/step
[*] 000004, d1[0.382] d2[0.452] g[31.376] ---> time[1.93], time_left[00:03:05]
1/1 [==============================] - 0s 22ms/step


[.] Saved Step : /content/Enhancement Model/Step Output/plot_000004.png
[.] Saved Model: /content/Enhancement Model/Model Output/gen_model_000004.h5
[.] Saved Model: /content/Enhancement Model/Model Output/disc_model_000004.h5
1/1 [==============================] - 0s 31ms/step
[*] 000005, d1[0.474] d2[0.418] g[35.578] ---> time[1.85], time_left[00:02:55]
1/1 [==============================] - 0s 23ms/step


...

[.] Saved Step : /content/Enhancement Model/Step Output/plot_000096.png
[.] Saved Model: /content/Enhancement Model/Model Output/gen_model_000096.h5
[.] Saved Model: /content/Enhancement Model/Model Output/disc_model_000096.h5
1/1 [==============================] - 0s 51ms/step
[*] 000097, d1[0.313] d2[0.391] g[25.786] ---> time[1.97], time_left[00:00:05]
1/1 [==============================] - 0s 35ms/step


[.] Saved Step : /content/Enhancement Model/Step Output/plot_000097.png
[.] Saved Model: /content/Enhancement Model/Model Output/gen_model_000097.h5
[.] Saved Model: /content/Enhancement Model/Model Output/disc_model_000097.h5
1/1 [==============================] - 0s 47ms/step
[*] 000098, d1[0.373] d2[0.324] g[28.114] ---> time[2.18], time_left[00:00:04]
1/1 [==============================] - 0s 21ms/step


[.] Saved Step : /content/Enhancement Model/Step Output/plot_000098.png
[.] Saved Model: /content/Enhancement Model/Model Output/gen_model_000098.h5
[.] Saved Model: /content/Enhancement Model/Model Output/disc_model_000098.h5
1/1 [==============================] - 0s 29ms/step
[*] 000099, d1[0.368] d2[0.331] g[22.474] ---> time[1.94], time_left[00:00:01]
1/1 [==============================] - 0s 23ms/step


[.] Saved Step : /content/Enhancement Model/Step Output/plot_000099.png
[.] Saved Model: /content/Enhancement Model/Model Output/gen_model_000099.h5
[.] Saved Model: /content/Enhancement Model/Model Output/disc_model_000099.h5
1/1 [==============================] - 0s 27ms/step
[*] 000100, d1[0.306] d2[0.422] g[19.277] ---> time[1.92], time_left[00:00:00]
1/1 [==============================] - 0s 22ms/step


[.] Saved Step : /content/Enhancement Model/Step Output/plot_000100.png
[.] Saved Model: /content/Enhancement Model/Model Output/gen_model_000100.h5
[.] Saved Model: /content/Enhancement Model/Model Output/disc_model_000100.h5


In [ ]:
import numpy as np
import cv2, os
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import img_to_array, save_img, array_to_img

model_path = "/content/Enhancement Model/Model Output/gen_model_000100.h5"
image_path = "/content/drive/MyDrive/r0a8b59bbt.png"

# Processing Image
img = cv2.imread(image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_arr = (img_to_array(img) - 127.5) / 127.5
resized = cv2.resize(img_arr, (256, 256), interpolation=cv2.INTER_AREA)
ready_img = np.expand_dims(resized, axis=0)

# Loading Model
model = load_model(model_path)

# Prdicting Image
pred = model.predict(ready_img)
pred = (cv2.medianBlur(pred[0], 1) + 1) / 2
pred = array_to_img(pred)

save_img("./output5.png", pred)






1/1 [==============================] - 0s 357ms/step
